In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader 
from torchvision import datasets, transforms 
from torchvision.utils import make_grid 


import numpy as np
import pandas as pd 
from sklearn.metrics import confusion_matrix 


In [2]:
transform=transforms.ToTensor()

In [3]:
train_data=datasets.MNIST(root='C:\\Users\\DELL\\OneDrive\\Desktop', train=True, download=False, transform=transform)

In [4]:
test_data=datasets.MNIST(root='C:\\Users\\DELL\\OneDrive\\Desktop', train=False, download=False, transform=transform)

In [5]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: C:\Users\DELL\OneDrive\Desktop
    Split: Train
    StandardTransform
Transform: ToTensor()

In [6]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: C:\Users\DELL\OneDrive\Desktop
    Split: Test
    StandardTransform
Transform: ToTensor()

In [7]:
train_loader=DataLoader(train_data, batch_size=10, shuffle=True)
test_loader=DataLoader(test_data, batch_size=10, shuffle=False)

In [8]:
conv1=nn.Conv2d(1,6,3,1) #1 input channel (because gray scale images), 6 output channels for feature extractions (number of filters), each 3*3 size, stride to be taken as 1
conv2=nn.Conv2d(6,16,3,1) #6 input channels (because output of above is 6), 16 output filters, 3*3 size of each filter, stride to be 1

In [9]:
for i, (X_train, y_train) in enumerate(train_data):
    break

In [10]:
X_train.shape

torch.Size([1, 28, 28])

In [11]:
x=X_train.view(1,1,28,28)

In [12]:
x=F.relu(conv1(x))

In [13]:
x.shape

torch.Size([1, 6, 26, 26])

In [14]:
#in the above, we got 26*26 instead of original 28*28 because the filters reduced the image size. no need to add padding for 
#mnist dataset because the numbers are in the middle of each image

In [15]:
x=F.max_pool2d(x,2,2) #input, kernel size, stride 

In [16]:
x.shape

torch.Size([1, 6, 13, 13])

In [17]:
#its 13*13 now because we're pooling 

In [18]:
x=F.relu(conv2(x))

In [19]:
x.shape

torch.Size([1, 16, 11, 11])

In [20]:
x=F.max_pool2d(x,2,2)

In [21]:
x.shape

torch.Size([1, 16, 5, 5])

In [22]:
(((28-2)/2)-2)/2

5.5

In [23]:
x.view(-1,16*5*5).shape

torch.Size([1, 400])

In [27]:
class ConvolutionalNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(1,6,3,1)
        self.conv2=nn.Conv2d(6,16,3,1)
        self.fc1=nn.Linear(5*5*16, 120)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10) #84 and 120 are arbitary values 
        
    def forward(self,X):
        X=F.relu(self.conv1(X))
        X=F.max_pool2d(X,2,2)
        X=F.relu(self.conv2(X))
        X=F.max_pool2d(X,2,2)
        X=X.view(-1,16*5*5)
        X=F.relu(self.fc1(X))
        X=F.relu(self.fc2(X))
        X=self.fc3(X)
        return F.log_softmax(X, dim=1)

In [28]:
torch.manual_seed(42)
model=ConvolutionalNetwork()

In [29]:
model

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [30]:
for param in model.parameters():
    print(param.numel())

54
6
864
16
48000
120
10080
84
840
10


In [31]:
criterion=nn.CrossEntropyLoss()

In [32]:
optimizer=torch.optim.Adam(model.parameters(), lr=0.001)

In [38]:
import time
start_time=time.time()

epochs=1
train_losses=[]
test_losses=[]
train_correct=[]
test_correct=[]

for i in range (epochs):
    trn_corr=0
    test_corr=0
    for b,(X_train, y_train) in enumerate(train_loader):
        b+=1
        y_pred=model(X_train) #no need to flatten here like ann because cnn accepts 2d data 
        loss=criterion(y_pred, y_train)
        
        predicted=torch.max(y_pred.data,1)[1]
        batch_corr=(predicted==y_train).sum()
        trn_corr+=batch_corr
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (b%600==0) :
            print(f"Epoch {i} Batch: {b} loss: {loss.item()}")
    train_losses.append(loss)
    train_correct.append(trn_corr)
    
    with torch.no_grad():
        for b,(X_test, y_test) in enumerate(test_loader):
            y_val=model(X_test)
            
            predicted=torch.max(y_val.data,1)[1]
            
            test_corr+=(predicted==y_test).sum()
            
    loss=criterion(y_val, y_test)
    test_losses.append(loss)
    test_correct.append(test_corr)



Epoch 0 Batch: 600 loss: 0.005933754611760378
Epoch 0 Batch: 1200 loss: 0.0010621579131111503
Epoch 0 Batch: 1800 loss: 4.831186743103899e-05
Epoch 0 Batch: 2400 loss: 0.00040188911953009665
Epoch 0 Batch: 3000 loss: 0.10219243913888931
Epoch 0 Batch: 3600 loss: 0.015040455386042595
Epoch 0 Batch: 4200 loss: 0.0003014873946085572
Epoch 0 Batch: 4800 loss: 0.04102698713541031
Epoch 0 Batch: 5400 loss: 0.0026962088886648417
Epoch 0 Batch: 6000 loss: 0.0009217123733833432


TypeError: 'float' object is not callable

In [41]:
cur_time=time.time()
total=cur_time-start_time
print(f"Training took {total/60} time")

Training took 10.63441452185313 time


In [42]:
test_load_all=DataLoader(test_data, batch_size=10000, shuffle=False)


In [43]:
with torch.no_grad():
    correct=0
    for X_test, y_test in test_load_all:
        y_val=model(X_test)
        predicted=torch.max(y_val,1)[1]
        correct+=(predicted==y_test).sum()

In [45]:
correct.item()/len(test_data)

0.9876

In [47]:
#how to test for a single image?
test_data[6904][0].reshape(28,28)

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000

In [48]:
model.eval()
with torch.no_grad():
    new_pred=model(test_data[6904][0].view(1,1,28,28))

In [50]:
new_pred.argmax()

tensor(8)